In [1]:
import os
import shutil
import pandas as pd
import yaml
from ultralytics import YOLO

In [2]:
import os
import shutil
import pandas as pd

# Define dataset paths
base_dir = os.path.abspath(".")  # Current working directory: D:\Sem 6 project
data_dir = os.path.join(base_dir, "MURA-v1.1")  # D:\Sem 6 project\MURA-v1.1
yolo_data_dir = os.path.join(base_dir, "MURA_YOLO")  # D:\Sem 6 project\MURA_YOLO

def prepare_yolo_dataset(csv_file, split, label_df):
    """ Converts dataset into YOLO format with improved debugging and label creation. """
    
    # Load CSV with correct column names
    df = pd.read_csv(csv_file, header=None, names=["image_path"])
    
    print(f"\n📊 CSV Analysis for {split}:")
    print(f"   - Total entries in CSV: {len(df)}")
    
    # Check for duplicate entries
    duplicates = df.duplicated().sum()
    print(f"   - Duplicate entries: {duplicates}")
    
    # Create YOLO directory structure
    images_dir = os.path.join(yolo_data_dir, split, "images")
    labels_dir = os.path.join(yolo_data_dir, split, "labels")
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    processed_count = 0
    missing_count = 0
    
    # Track unique filenames to avoid duplicates
    copied_filenames = set()
    
    # Map studies to labels
    study_to_label = dict(zip(label_df["study_path"], label_df["label"]))
    
    # Define body parts and class mapping
    body_parts = ["XR_ELBOW", "XR_FINGER", "XR_FOREARM", "XR_HAND", "XR_HUMERUS", "XR_SHOULDER", "XR_WRIST"]
    class_names = [f"{bp}_{cond}" for bp in body_parts for cond in ["negative", "positive"]]
    class_to_id = {name: idx for idx, name in enumerate(class_names)}

    for index, row in df.iterrows():
        # Construct full image path
        image_rel_path = row["image_path"]  # e.g., MURA-v1.1/train/XR_SHOULDER/...
        image_path = os.path.join(base_dir, image_rel_path)  # D:\Sem 6 project\MURA-v1.1\train\...
        image_path = os.path.normpath(image_path)
        
        filename = os.path.basename(image_path)
        
        if index < 5:
            print(f"🔹 Checking file: {image_path}")
            print(f"   - Exists: {os.path.exists(image_path)}")
        
        if not os.path.exists(image_path):
            if missing_count < 10:
                print(f"⚠️ Missing file: {image_path}")
            missing_count += 1
            continue  

        if filename in copied_filenames:
            base, ext = os.path.splitext(filename)
            filename = f"{base}_{index}{ext}"
        
        dest_image_path = os.path.join(images_dir, filename)
        copied_filenames.add(filename)
        
        shutil.copy(image_path, dest_image_path)
        processed_count += 1
        
        # Create label file
        # Get study path from image path
        study_path = "/".join(image_rel_path.split("/")[:-1]) + "/"  # e.g., MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/
        label = study_to_label.get(study_path, None)
        if label is None:
            print(f"Warning: No label found for study {study_path}")
            continue
        
        # Extract body part from image path
        body_part = [bp for bp in body_parts if bp in image_rel_path][0]
        condition = "positive" if label == 1 else "negative"
        class_name = f"{body_part}_{condition}"
        class_id = class_to_id[class_name]
        
        lbl_name = filename.replace(".png", ".txt")
        dest_label_path = os.path.join(labels_dir, lbl_name)
        with open(dest_label_path, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
        
        if processed_count % 1000 == 0:
            print(f"✅ {processed_count} images processed...")

    print(f"📸 Total {processed_count} images processed for {split}.")
    print(f"🚨 Total {missing_count} missing images in {split} set.")

print("🔄 Preparing YOLO dataset...")
print(f"Current working directory: {os.getcwd()}")

if not os.path.exists(data_dir):
    print(f"❌ Data directory not found: {data_dir}")
    exit(1)

train_csv = os.path.join(data_dir, "train_image_paths.csv")
valid_csv = os.path.join(data_dir, "valid_image_paths.csv")
train_label_df = pd.read_csv(os.path.join(data_dir, "train_labeled_studies.csv"), header=None, names=["study_path", "label"])
valid_label_df = pd.read_csv(os.path.join(data_dir, "valid_labeled_studies.csv"), header=None, names=["study_path", "label"])

if not os.path.exists(train_csv):
    print(f"❌ Training CSV file not found: {train_csv}")
else:
    print(f"✅ Training CSV found: {train_csv}")
    try:
        train_df = pd.read_csv(train_csv, header=None, nrows=5)
        print("Sample training CSV content:")
        print(train_df)
    except Exception as e:
        print(f"Error reading CSV: {e}")
    prepare_yolo_dataset(train_csv, "train", train_label_df)

if not os.path.exists(valid_csv):
    print(f"❌ Validation CSV file not found: {valid_csv}")
else:
    print(f"✅ Validation CSV found: {valid_csv}")
    prepare_yolo_dataset(valid_csv, "valid", valid_label_df)

print("✅ Dataset preparation complete!")

🔄 Preparing YOLO dataset...
Current working directory: D:\Sem 6 project
✅ Training CSV found: D:\Sem 6 project\MURA-v1.1\train_image_paths.csv
Sample training CSV content:
                                                   0
0  MURA-v1.1/train/XR_SHOULDER/patient00001/study...
1  MURA-v1.1/train/XR_SHOULDER/patient00001/study...
2  MURA-v1.1/train/XR_SHOULDER/patient00001/study...
3  MURA-v1.1/train/XR_SHOULDER/patient00002/study...
4  MURA-v1.1/train/XR_SHOULDER/patient00002/study...

📊 CSV Analysis for train:
   - Total entries in CSV: 36808
   - Duplicate entries: 0
🔹 Checking file: D:\Sem 6 project\MURA-v1.1\train\XR_SHOULDER\patient00001\study1_positive\image1.png
   - Exists: True
🔹 Checking file: D:\Sem 6 project\MURA-v1.1\train\XR_SHOULDER\patient00001\study1_positive\image2.png
   - Exists: True
🔹 Checking file: D:\Sem 6 project\MURA-v1.1\train\XR_SHOULDER\patient00001\study1_positive\image3.png
   - Exists: True
🔹 Checking file: D:\Sem 6 project\MURA-v1.1\train\XR_SHOULDER\pa

KeyboardInterrupt: 

In [3]:
from ultralytics import YOLO
import torch

# Load YOLO model
model = YOLO("yolov8n.pt")

# Train with checkpoint saving after every epoch
model.train(
    data="MURA_YOLO/mura.yaml",  # Dataset YAML file
    epochs=25,                   # Number of epochs
    imgsz=640,                   # Image size
    batch=16,                     # Batch size
    project="runs/train",        # Project directory
    name="mura_yolov8(25 epoch)7", # Run name
    device=0 if torch.cuda.is_available() else "cpu",  # Use GPU if available
    save=True,                    # Enable checkpoint saving
    save_period=1                  # Save model weights after every epoch
)
model.resume()

New https://pypi.org/project/ultralytics/8.3.96 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.94  Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=MURA_YOLO/mura.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=0, workers=8, project=runs/train, name=mura_yolov8(25 epoch)73, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None

train: Scanning D:\Sem 6 project\MURA_YOLO\train\labels... 36806 images, 0 backgrounds, 2 corrupt: 100%|██████████| 368

train: WARNING  D:\Sem 6 project\MURA_YOLO\train\images\image1_10713.png: ignoring corrupt image/label: cannot identify image file 'D:\\Sem 6 project\\MURA_YOLO\\train\\images\\image1_10713.png'
train: WARNING  D:\Sem 6 project\MURA_YOLO\train\images\image1_488.png: ignoring corrupt image/label: cannot identify image file 'D:\\Sem 6 project\\MURA_YOLO\\train\\images\\image1_488.png'


train: New cache created: D:\Sem 6 project\MURA_YOLO\train\labels.cache


val: Scanning D:\Sem 6 project\MURA_YOLO\valid\labels.cache... 3197 images, 0 backgrounds, 0 corrupt: 100%|██████████| 


...

                   all       3197       3197      0.803      0.818      0.867      0.867



25 epochs completed in 2.042 hours.
Optimizer stripped from runs\train\mura_yolov8(25 epoch)73\weights\last.pt, 6.2MB
Optimizer stripped from runs\train\mura_yolov8(25 epoch)73\weights\best.pt, 6.2MB

Validating runs\train\mura_yolov8(25 epoch)73\weights\best.pt...
Ultralytics 8.3.94  Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 100/100 [00:


                   all       3197       3197      0.804      0.817      0.867      0.867
     XR_ELBOW_negative        235        235      0.748      0.962      0.883      0.883
     XR_ELBOW_positive        230        230       0.87      0.756      0.904      0.904
    XR_FINGER_negative        214        214      0.707      0.893      0.814      0.814
    XR_FINGER_positive        247        247      0.865       0.75        0.9        0.9
   XR_FOREARM_negative        150        150      0.735      0.852      0.848      0.848
   XR_FOREARM_positive        151        151      0.853      0.615      0.833      0.833
      XR_HAND_negative        271        271      0.742      0.956      0.874      0.874
      XR_HAND_positive        189        189      0.813      0.603      0.804      0.804
   XR_HUMERUS_negative        148        148      0.875      0.797      0.889      0.889
   XR_HUMERUS_positive        140        140      0.844      0.893      0.902      0.902
  XR_SHOULDER_negativ

AttributeError: 'DetectionModel' object has no attribute 'resume'

In [4]:
import os

valid_labels_dir = "D:/Sem 6 project/MURA_YOLO/valid/labels"
y_true = []

# Iterate through label files
for label_file in os.listdir(valid_labels_dir):
    if label_file.endswith(".txt"):
        with open(os.path.join(valid_labels_dir, label_file), "r") as f:
            line = f.readline().strip()
            class_id = int(line.split()[0])  # Extract class ID (first value)
            y_true.append(class_id)

print(f"Extracted {len(y_true)} true labels.")

Extracted 3197 true labels.


In [11]:
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, roc_curve, precision_recall_curve, matthews_corrcoef, confusion_matrix, log_loss
)
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os
import torch

# Clear GPU memory
torch.cuda.empty_cache()

# Paths
VALID_IMAGES_DIR = "D:/Sem 6 project/MURA_YOLO/valid/images"
VALID_LABELS_DIR = "D:/Sem 6 project/MURA_YOLO/valid/labels"
MODEL_PATH = "D:/Sem 6 project/runs/train/mura_yolov8(25 epoch)7/weights/best.pt"

# Step 1: Extract ground truth
y_true = []
label_files = sorted([f for f in os.listdir(VALID_LABELS_DIR) if f.endswith(".txt")])
for label_file in label_files:
    with open(os.path.join(VALID_LABELS_DIR, label_file), "r") as f:
        class_id = int(f.readline().strip().split()[0])
        y_true.append(class_id)

# Step 2: Extract predictions and scores
model = YOLO(MODEL_PATH)
image_files = sorted([os.path.join(VALID_IMAGES_DIR, f) for f in os.listdir(VALID_IMAGES_DIR) if f.endswith(".png")])
batch_size = 8
y_pred = []
y_scores = []
y_scores_full = []

for i in range(0, len(image_files), batch_size):
    batch_files = image_files[i:i + batch_size]
    results = model.predict(batch_files, save=False, imgsz=640, conf=0.1, device="cuda" if torch.cuda.is_available() else "cpu", verbose=False)
    
    for result in results:
        if len(result.boxes) > 0:
            box = result.boxes[0]  # Highest confidence detection
            pred_class = int(box.cls.item())
            pred_score = box.conf.item()
            y_pred.append(pred_class)
            y_scores.append(pred_score)
            scores = [pred_score if j == pred_class else (1 - pred_score) / (13) for j in range(14)]
            y_scores_full.append(scores / np.sum(scores))
        else:
            print(f"Warning: No detection for image {result.path}")
            y_pred.append(0)
            y_scores.append(0.0)
            y_scores_full.append([1.0 / 14] * 14)
    
    torch.cuda.empty_cache()

# Verify lengths
print(f"Length of y_true: {len(y_true)}, Length of y_pred: {len(y_pred)}")
assert len(y_true) == len(y_pred), "Mismatch in lengths! Check image-label pairing."

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)

# Explicitly extract scalar values
precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
precision = precision.item() if hasattr(precision, 'item') else float(precision)

recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
recall = recall.item() if hasattr(recall, 'item') else float(recall)

f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
f1 = f1.item() if hasattr(f1, 'item') else float(f1)

kappa = cohen_kappa_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)

# Specificity and G-mean
cm = confusion_matrix(y_true, y_pred)
specificities = []
for i in range(cm.shape[0]):
    tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
    fp = cm[:, i].sum() - cm[i, i]
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificities.append(specificity)
specificity_avg = np.mean(specificities)
g_mean = np.sqrt(recall * specificity_avg)

# ROC AUC and Log Loss
n_classes = 14
class_names = ['XR_ELBOW_neg', 'XR_ELBOW_pos', 'XR_FINGER_neg', 'XR_FINGER_pos', 
               'XR_FOREARM_neg', 'XR_FOREARM_pos', 'XR_HAND_neg', 'XR_HAND_pos',
               'XR_HUMERUS_neg', 'XR_HUMERUS_pos', 'XR_SHOULDER_neg', 'XR_SHOULDER_pos',
               'XR_WRIST_neg', 'XR_WRIST_pos']
y_true_bin = np.array([np.eye(n_classes)[label] for label in y_true])
roc_auc = roc_auc_score(y_true_bin, y_scores_full, multi_class="ovr", average="macro")
logloss = log_loss(y_true_bin, y_scores_full)

# ROC Curves
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], [score[i] for score in y_scores_full])
    plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc_score(y_true_bin[:, i], [score[i] for score in y_scores_full]):.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves (One-vs-Rest)')
plt.legend(loc='best', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

# PR Curves
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    precision, recall_pr, _ = precision_recall_curve(y_true_bin[:, i], [score[i] for score in y_scores_full])
    plt.plot(recall_pr, precision, label=f'{class_names[i]}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves (One-vs-Rest)')
plt.legend(loc='best', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

# Print metrics
print(f"Accuracy: {accuracy:.4f}")

print(f"Specificity (Avg): {specificity_avg:.4f}")
print(f"G-Mean: {g_mean:.4f}")
print(f"AUC (ROC-AUC): {roc_auc:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")
print(f"Log Loss: {logloss:.4f}")

Length of y_true: 3197, Length of y_pred: 3197


<Figure size 1000x800 with 1 Axes>

<Figure size 1000x800 with 1 Axes>

Accuracy: 0.5837
Specificity (Avg): 0.9676
G-Mean: 0.7322
AUC (ROC-AUC): 0.8115
MCC: 0.5553
Cohen's Kappa: 0.5469
Log Loss: 1.7554


In [14]:
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, roc_curve, precision_recall_curve, matthews_corrcoef, confusion_matrix, log_loss
)
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os
import torch

# Clear GPU memory
torch.cuda.empty_cache()

# Paths
VALID_IMAGES_DIR = "D:/Sem 6 project/MURA_YOLO/valid/images"
VALID_LABELS_DIR = "D:/Sem 6 project/MURA_YOLO/valid/labels"
MODEL_PATH = "D:/Sem 6 project/runs/train/mura_yolov8(25 epoch)7/weights/best.pt"

# Step 1: Extract ground truth
y_true = []
label_files = sorted([f for f in os.listdir(VALID_LABELS_DIR) if f.endswith(".txt")])
for label_file in label_files:
    with open(os.path.join(VALID_LABELS_DIR, label_file), "r") as f:
        class_id = int(f.readline().strip().split()[0])
        y_true.append(class_id)

# Step 2: Extract predictions and scores
model = YOLO(MODEL_PATH)
image_files = sorted([os.path.join(VALID_IMAGES_DIR, f) for f in os.listdir(VALID_IMAGES_DIR) if f.endswith(".png")])
batch_size = 8
y_pred = []
y_scores = []
y_scores_full = []

for i in range(0, len(image_files), batch_size):
    batch_files = image_files[i:i + batch_size]
    results = model.predict(batch_files, save=False, imgsz=640, conf=0.1, device="cuda" if torch.cuda.is_available() else "cpu", verbose=False)
    
    for result in results:
        if len(result.boxes) > 0:
            box = result.boxes[0]  # Highest confidence detection
            pred_class = int(box.cls.item())
            pred_score = box.conf.item()
            y_pred.append(pred_class)
            y_scores.append(pred_score)
            scores = [pred_score if j == pred_class else (1 - pred_score) / (13) for j in range(14)]
            y_scores_full.append(scores / np.sum(scores))
        else:
            print(f"Warning: No detection for image {result.path}")
            y_pred.append(0)
            y_scores.append(0.0)
            y_scores_full.append([1.0 / 14] * 14)
    
    torch.cuda.empty_cache()

# Verify lengths
print(f"Length of y_true: {len(y_true)}, Length of y_pred: {len(y_pred)}")
assert len(y_true) == len(y_pred), "Mismatch in lengths! Check image-label pairing."

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)

# Explicitly convert to float
precision = float(precision_score(y_true, y_pred, average="macro", zero_division=0))
recall = float(recall_score(y_true, y_pred, average="macro", zero_division=0))
f1 = float(f1_score(y_true, y_pred, average="macro", zero_division=0))
kappa = cohen_kappa_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)

# Specificity and G-mean
cm = confusion_matrix(y_true, y_pred)
specificities = []
for i in range(cm.shape[0]):
    tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
    fp = cm[:, i].sum() - cm[i, i]
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificities.append(specificity)
specificity_avg = np.mean(specificities)
g_mean = np.sqrt(recall * specificity_avg)

# ROC AUC and Log Loss
n_classes = 14
class_names = ['XR_ELBOW_neg', 'XR_ELBOW_pos', 'XR_FINGER_neg', 'XR_FINGER_pos', 
               'XR_FOREARM_neg', 'XR_FOREARM_pos', 'XR_HAND_neg', 'XR_HAND_pos',
               'XR_HUMERUS_neg', 'XR_HUMERUS_pos', 'XR_SHOULDER_neg', 'XR_SHOULDER_pos',
               'XR_WRIST_neg', 'XR_WRIST_pos']
y_true_bin = np.array([np.eye(n_classes)[label] for label in y_true])
roc_auc = float(roc_auc_score(y_true_bin, y_scores_full, multi_class="ovr", average="macro"))
logloss = float(log_loss(y_true_bin, y_scores_full))

# ROC Curves
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], [score[i] for score in y_scores_full])
    plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {float(roc_auc_score(y_true_bin[:, i], [score[i] for score in y_scores_full])):.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves (One-vs-Rest)')
plt.legend(loc='best', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

# PR Curves
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    precision, recall_pr, _ = precision_recall_curve(y_true_bin[:, i], [score[i] for score in y_scores_full])
    plt.plot(recall_pr, precision, label=f'{class_names[i]}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves (One-vs-Rest)')
plt.legend(loc='best', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Specificity (Avg): {specificity_avg:.4f}")
print(f"G-Mean: {g_mean:.4f}")
print(f"AUC (ROC-AUC): {roc_auc:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")
print(f"Log Loss: {logloss:.4f}")

Length of y_true: 3197, Length of y_pred: 3197


<Figure size 1000x800 with 1 Axes>

<Figure size 1000x800 with 1 Axes>

Accuracy: 0.5837
Precision: [   0.092274    0.092303    0.092332 ...           1           1           1]
Recall: 0.5539817196650395
F1 Score: 0.5175998154250524
Specificity (Avg): 0.9676
G-Mean: 0.7322
AUC (ROC-AUC): 0.8115
MCC: 0.5553
Cohen's Kappa: 0.5469
Log Loss: 1.7554
